In [7]:
import math as mat
import pandas as pd
import numpy as np

In [11]:
draftpool = pd.read_csv("C:/Users/night/draftpool.csv")
drafted = pd.read_csv("https://atl-01.statsplus.net/flg/api/draft")

# parks <- read.csv("parkfactors.csv")
# leagues <- read.csv("leaguefactors.csv")

# draftpool <- draftpool[!(draftpool$ID %in% drafted$ID), ]

# lg_obp <- 0.317366582
# lg_slg <- 0.424479901
# rs_scale <- 1.25
# er_pct <- 0.94
# lg_era <- 4.425
# lg_ra9 <- 4.70
# avg_scale <- 1.034
# run_sb <- 0.23
# run_cs <- -0.5
# run_fb <- -0.003
# run_pa <- 0.025
# c_fip <- 3.32
# warip <- 0.000867
# rppa <- 0.125
# rpw <- 10.0


In [4]:
def conv250(x):
    return mat.ceil(((x + 2/5) -20) * 200 / (80-20)-1)

def conv20_80(x):
    grade = (x  * (80 - 20) / 200) + 20
    if grade < 20:
        grade = 20
    return grade

def catcher(c_ability, c_arm):
    ability = (conv250(c_ability) - 125) / 25 * 19.5
    arm = (conv250(c_arm) - 125) / 25 * 19.5
    return c_ability + arm +133

def first(height, if_range, if_error, if_arm, turn_dp):
    if height > 215:
        height = 215
    infE = conv250(if_error)
    infR = conv250(if_range)
    infA = conv250(if_arm)
    infD = conv250(turn_dp)
    if infR < 90:
        infR = infR / 3
    else:
        infR = 30 + (infR - 90) * 2 / 25
    if infE < 90:
        infE = infE / 5
    else:
        infE = 18 + (infE - 90) / 25
    infA = infA / 70
    infD = infD / 70
    heightFactor = 1 + (height - 155) / 15
    return heightFactor * (infA + infD + infE + infR)


def second(if_range, if_error, if_arm, turn_dp):
    if_range = (conv250(if_range) - 125) / 25 * 21.5
    error = (conv250(if_error) - 125) / 25 * 8.25
    arm = (conv250(if_arm) - 125) / 25 * 1.5
    turn = (conv250(turn_dp) - 125) / 25 * 9
    return if_range + error + arm + turn + 129


def third(if_range, if_error, if_arm, turn_dp):
    if_range = (conv250(if_range) - 125) / 25 * 13
    error = (conv250(if_error) - 125) / 25 * 7.5
    arm = (conv250(if_arm) - 125) / 25 * 17
    turn = (conv250(turn_dp) - 125) / 25 * 3.25
    return if_range + error + arm + turn + 113.5

def short(if_range, if_error, if_arm, turn_dp):
    if_range = (conv250(if_range) - 125) / 25 * 25.5
    error = (conv250(if_error) - 125) / 25 * 7.5
    arm = (conv250(if_arm) - 125) / 25 * 2
    turn = (conv250(turn_dp) - 125) / 25 * 8
    return if_range + error + arm + turn + 93.75


def left(of_range, of_error, of_arm):
    of_range <- (conv250(of_range) - 125) / 25 * 29.5
    error <- (conv250(of_error) - 125) / 25 * 4
    arm <- (conv250(of_arm) - 125) / 25 * 5.75
    return of_range + error + arm + 149

def center(of_range, of_error, of_arm):
    of_range <- (conv250(of_range) - 125) / 25 * 43
    error <- (conv250(of_error) - 125) / 25 * 3.5
    arm <- (conv250(of_arm) - 125) / 25 * 1.75
    return of_range + error + arm + 178

def right(of_range, of_error, of_arm):
    of_range <- (conv250(of_range) - 125) / 25 * 25.5
    error <- (conv250(of_error) - 125) / 25 * 4
    arm <- (conv250(of_arm) - 125) / 25 * 11
    return of_range + error + arm + 129



In [76]:

def calcDWAR(rating, position):
    grade = conv20_80(rating)
    slopes = [0.003, 0.009, 0.018, 0.015, 0.024, 0.009, 0.015, 0.009]
    intercepts = [1.1,-1.75,-1.5,-1.05,-1.8,-1.3,-1.15,-1.25]
    i = position - 1
    if grade > 20:
        war = round((slopes[i] * rating + intercepts[i]) * 10,0) / 10
        return war
    else:
        return print('NA')
    
def calcOWar(contact, gap, power, eye, avoid_k, speed, stealing, bats):
    mlo = [0.0009, 0.18, 0.18, 0.48, -1.2, 0.0006, 0.0009, 0.003]
    blo = [0.16, 11, 0, 2, 260, 0.04, 0, 0.35]
    mhi = [0.0006, 0.18, 0.3, 0.72, -0.9, 0.0003, 0.0018, 0.0018]
    bhi = [0.19, 11, -12, -22, 230, 0.07, -0.09, 0.47]
    parks = parks[23,]
    rates = [contact, gap, power, eye, avoid_k, speed, stealing, bats]
    stats = []
  
    for i in range(len(rates)):
        rate = rates[i]

        if rate < 100:
            states = stats.append(rate * mlo[i] + blo [i])
        else:
            states = stats.append(rate * mhi[i] + bhi [i])
  
    pa = 600
    mult = pa / (550 + stats[4])
    bb = stats[4] * mult
    ab = pa - bb
    
    if grepl('L', bats):
        avg = stats[1] * park['AVGL']
    elif grepl('R', bats):
        avg = stats[1] * park['AVGR']
    else:
        avg = stats[1] * park['AVGS']
  
  
    h <- avg * ab
    h2 <- stats[2] * (1 - stats[6]) * park['Doubles'] * mult
    h3 <- stats[2] * (stats[6]) * park['Triples'] * mult
  
    if grepl('L', bats):
        hr = max(stats[2], 1) * park['HRL'] * mult
    elif grepl('R', bats):
        hr = max(stats[2], 1) * park['HRR'] * mult
    else:
        hr = max(stats[2], 1) * park['HRS'] * mult
 
    obp <- (h + bb) / pa
    slg <- (h + h2 + 2 * h3 + 3 * hr) / ab
    opsp <- (obp / lg_obp + slg / lg_slg - 1) * 100 / park['OPS']
    fb <- h - h2 - h3 - hr + bb
    sba <- stats[7] * fb
    sb <- stats[8] * sba
    cs <- sba - sb
    bat <- (opsp / 100 - 1) * rppa * pa
    bsr <- run_sb * sb + run_cs * cs + run_fb * fb
    return round((bat + bsr + pa * run_pa) / rpw,2)

calcPWAR <-
  function(stuff,
           movement,
           control,
           stamina,
           hold,
           gbfb,
           position) {
    league <- leagues[2, ]
    park <- parks[23, ]
    stats <- c()
    if (gbfb >= 64) {
      stats <- stats %>% append(0.28)
    } else if (gbfb >= 59 && gbfb <= 63) {
      stats <- stats %>% append(0.285)
    } else if (gbfb >= 44 && gbfb <= 48) {
      stats <- stats %>% append(0.295)
    } else if (gbfb <= 43) {
      stats <- stats %>% append(0.3)
    } else {
      stats <- stats %>% append(0.29)
    }
    
    stats <- stats %>% append(0.9 * stuff + 30)
    stats <- stats %>% append(max(-0.12 * movement + 32, 2))
    
    if (control < 100) {
      stats <- stats %>% append(-0.84 * control + 140)
    } else {
      stats <- stats %>% append(max(-0.42 * control + 98, 7))
    }
    
    if (grepl("SP", position)) {
      if (stamina < 100) {
        stats <- stats %>% append(0.06 * stamina + 15)
      } else {
        stats <- stats %>% append(0.03 * stamina + 18)
      }
    } else {
      if (stamina < 100) {
        stats <- stats %>% append(0.024 * stamina + 3.6)
      } else {
        stats <- stats %>% append(0.03 * stamina + 3)
      }
    }
    
    if (hold < 100) {
      stats <- stats %>% append(-0.012 * hold + 1.2)
    } else {
      stats <- stats %>% append(-0.06 * hold + 0.6)
    }
    
    gp <- 0
    gs <- 0
    
    if (grepl("SP", position)) {
      gp <- 32
      gs <- 32
    } else {
      gp <- 64
    }
    
    
    ab <- gp * stats[5]
    
    mult <- ab / 550
    
    babip <- stats[1] * league$BABIP
    so <- stats[2] * mult * league$SO
    hr <- stats[3] * mult * league$HR * park$HR
    bb <- stats[4] * mult * league$BB
    ha <-
      (babip * (550 - stats[2] - stats[3]) + stats[3]) * park$AVG * mult
    
    ip <- (ab - ha) / 3
    avg <- ha / ab
    rspct <- rs_scale * avg
    ra <- rspct * (ha - hr + bb) + hr + stats[5]
    er <- ra * er_pct * league$ERA
    era <- er / ip * 9
    davg <- ha / ab * avg_scale * 1000
    dbabip <- (ha - hr) / (ab - hr - so) * avg_scale * 1000
    whip <- (ha + bb) / ip
    
    hr9 <- hr / ip * 9
    bb9 <- bb / ip * 9
    k9 <- so / ip * 9
    kbb <- so / bb
    erap <- lg_era / (era) * 100 * league$ERA * park$ERA
    fip <- (13 * hr + 3 * bb - 2 * so) / ip + c_fip
    
    pfipr9 <- (fip / park$FIP + (lg_ra9 - lg_era))
    
    ippg <- ip / gp
    
    drpw <- ((((18 - ippg) * (lg_ra9 * league$ERA) +
                 (ippg * pfipr9)
    ) / 18) + 2) * 1.5
    repl <- 0.03 * (1 - gs / gp) + 0.12 * (gs / gp)
    war <-
      ((lg_ra9 * league$ERA - pfipr9) / drpw + repl) * ip / 9 + warip * ip
    
    imod <- ip %% 1 / 0.3
    
    if (imod > 2) {
      imod <- 0
      
    }
    
    war <- round(war, 2)
    
    return(war)
  }

-0.8

In [66]:

for (i in 1:nrow(draftpool)) {
  draftpool$potCRating[i] <-
    calcCatcher(draftpool$CAb[i], draftpool$CArm[i])
  draftpool$pot1BRating[i] <-
    calc1B(
      draftpool$Height[i],
      draftpool$IFR[i],
      draftpool$IFE[i],
      draftpool$IFA[i],
      draftpool$TDP[i]
    )
  draftpool$pot2BRating[i] <-
    calc2B(draftpool$IFR[i],
           draftpool$IFE[i],
           draftpool$IFA[i],
           draftpool$TDP[i])
  draftpool$pot3BRating[i] <-
    calc3B(draftpool$IFR[i],
           draftpool$IFE[i],
           draftpool$IFA[i],
           draftpool$TDP[i])
  draftpool$potSSRating[i] <-
    calcSS(draftpool$IFR[i],
           draftpool$IFE[i],
           draftpool$IFA[i],
           draftpool$TDP[i])
  draftpool$potLFRating[i] <-
    calcLF(draftpool$OFR[i], draftpool$OFE[i],
           draftpool$OFA[i])
  draftpool$potCFRating[i] <-
    calcLF(draftpool$OFR[i], draftpool$OFE[i],
           draftpool$OFA[i])
  draftpool$potRFRating[i] <-
    calcLF(draftpool$OFR[i], draftpool$OFE[i],
           draftpool$OFA[i])
  
  draftpool$potCGrade[i] <- conv2080(draftpool$potCRating[i])
  draftpool$pot1BGrade[i] <- conv2080(draftpool$pot1BRating[i])
  draftpool$pot2BGrade[i] <- conv2080(draftpool$pot2BRating[i])
  draftpool$pot3BGrade[i] <- conv2080(draftpool$pot3BRating[i])
  draftpool$potSSGrade[i] <- conv2080(draftpool$potSSRating[i])
  draftpool$potLFGrade[i] <- conv2080(draftpool$potLFRating[i])
  draftpool$potCFGrade[i] <- conv2080(draftpool$potCFRating[i])
  draftpool$potRFGrade[i] <- conv2080(draftpool$potRFRating[i])
  
  draftpool$WARC[i] <- calcDWAR(draftpool$potCRating[i], 2)
  draftpool$WAR1B[i] <- calcDWAR(draftpool$pot1BRating[i], 3)
  draftpool$WAR2B[i] <- calcDWAR(draftpool$pot2BRating[i], 4)
  draftpool$WAR3B[i] <- calcDWAR(draftpool$pot3BRating[i], 5)
  draftpool$WARSS[i] <- calcDWAR(draftpool$potSSRating[i], 6)
  draftpool$WARLF[i] <- calcDWAR(draftpool$potLFRating[i], 7)
  draftpool$WARCF[i] <- calcDWAR(draftpool$potCFRating[i], 8)
  draftpool$WARRF[i] <- calcDWAR(draftpool$potRFRating[i], 9)
  
  draftpool$oWAR[i] <- calcOWAR(
    conv250(draftpool$PotCntct[i]),
    conv250(draftpool$PotGap[i]),
    conv250(draftpool$PotPow[i]),
    conv250(draftpool$PotEye[i]),
    conv250(draftpool$PotKs[i]),
    conv250(draftpool$Speed[i]),
    conv250(draftpool$Steal[i]),
    draftpool$Bats[i]
  )
  
  draftpool$pWAR[i] <- calcPWAR(
    conv250(draftpool$PotStf[i]),
    conv250(draftpool$PotMov[i]),
    conv250(draftpool$PotCtrl[i]),
    conv250(draftpool$Stm[i]),
    conv250(draftpool$Hold[i]),
    draftpool$GB[i],
    draftpool$Pos[i]
  )
  
  if (grepl("SP", draftpool$Pos[i]) |
      grepl("RP", draftpool$Pos[i]) |
      grepl("CL", draftpool$Pos[i])){
    
    draftpool$WARprimary[i] <- 0
    draftpool$totalWAR[i] <- draftpool$pWAR[i]
    
    if (draftpool$oWAR[i] > 0){
      draftpool$totalWAR[i] <- draftpool$totalWAR[i] + draftpool$oWAR[i]
    }
    
  } else{
    if (grepl("C", draftpool$Pos[i])) {
      draftpool$WARprimary[i] <- draftpool$WARC[i]
    }
    else if (grepl("1B", draftpool$Pos[i])) {
      draftpool$WARprimary[i] <- draftpool$WAR1B[i]
    }
    else if (grepl("2B", draftpool$Pos[i])) {
      draftpool$WARprimary[i] <- draftpool$WAR2B[i]
    }
    else if (grepl("3B", draftpool$Pos[i])) {
      draftpool$WARprimary[i] <- draftpool$WAR3B[i]
    }
    else if (grepl("SS", draftpool$Pos[i])) {
      draftpool$WARprimary[i] <- draftpool$WARSS[i]
    }
    else if (grepl("LF", draftpool$Pos[i])) {
      draftpool$WARprimary[i] <- draftpool$WARLF[i]
    }
    else if (grepl("CF", draftpool$Pos[i])) {
      draftpool$WARprimary[i] <- draftpool$WARCF[i]
    }
    else if (grepl("RF", draftpool$Pos[i])) {
      draftpool$WARprimary[i] <- draftpool$WARRF[i]
    }
    
    if (draftpool$pWAR[i] > 0){
      draftpool$totalWAR[i] <-
        draftpool$oWAR[i] + draftpool$WARprimary[i] + draftpool$pWAR[i]
    } else {
      draftpool$totalWAR[i] <- draftpool$oWAR[i] + draftpool$WARprimary[i]
    }
    
  }
  
}

draftlist <-
  head(draftpool[order(-draftpool$totalWAR), 
                 c(1, 2, 3, 104, 105, 106, 107)], 200)
head(draftlist, 10)
write.csv(draftlist, "draftlist.csv", row.names = FALSE)

124.40152380952382